In [2]:
import warnings

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn import preprocessing

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [5]:
class LSTM(nn.Module):
  def __init__(self, num_classes, num_layers, input_size, hidden_size, device):
    super().__init__()
    self.num_classes = num_classes
    self.num_layer = num_layers
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.device = device
    
    self.lstm = nn.LSTM(
      input_size=input_size,
      hidden_size=hidden_size,
      num_layers=num_layers,
      batch_first=True,
      dropout=0.2)
    self.fc_1 = nn.Linear(hidden_size, 128)
    self.fc_2 = nn.Linear(128, num_classes)
    self.relu = nn.ReLU(inplace=True)
    
  def forward(self, x):
    h_0 = torch.zeros(self.num_layer, x.size(0), self.hidden_size, device=self.device)
    c_0 = torch.zeros(self.num_layer, x.size(0), self.hidden_size, device=self.device)
    
    output, (hn, cn) = self.lstm(x, (h_0, c_0))
    hn = hn.view(-1, self.hidden_size)
    out = self.relu(hn)
    out = self.fc_1(out)
    out = self.relu(out)
    out = self.fc_2(out)
    return out

In [29]:
class Predictor:
  def __init__(self):
    # 미리 정의된 거리의 역수를 통한 가중치
    self.near_aws = {
      "아름동": [('세종고운', 0.5959398623781422), ('세종금남', 0.20935575535139192), ('세종연서', 0.19470438227046594)],
      "신흥동":  [('세종연서', 0.623771911782061), ('세종고운', 0.23437985962111915), ('세종전의', 0.14184822859681984)],
      "노은동": [('계룡', 0.37215720734940244), ('세종금남', 0.31442215528240247), ('오월드', 0.313420637368195)],
      "문창동": [('오월드', 0.43792606175001325), ('세천', 0.36840926563842336), ('장동', 0.19366467261156328)],
      "읍내동": [('장동', 0.4600092574298802), ('세천', 0.29439745524012173), ('오월드', 0.2455932873299979)],
      "정림동": [('오월드', 0.6611945451443386), ('계룡', 0.17462776824421944), ('세천', 0.16417768661144194)],
      "공주": [('공주', 0.595096232672347), ('정안', 0.21560028832884157), ('세종금남', 0.18930347899881153)],
      "논산": [('논산', 0.8106894830885594), ('계룡', 0.10161496038210217), ('양화', 0.0876955565293385)],
      "대천2동": [('대천항', 0.5364383656912503), ('청양', 0.24341489240638425), ('춘장대', 0.22014674190236547)],
      "독곶리": [('대산', 0.805639758960593), ('안도', 0.10337714448943511), ('당진', 0.090983096549972)],
      "동문동": [('태안', 0.42383826452678564), ('당진', 0.3223343628472369), ('홍북', 0.2538273726259775)],
      "모종동": [('아산', 0.3665049233697947), ('성거', 0.33423205479638257), ('예산', 0.29926302183382286)],
      "신방동": [('성거', 0.4233387535096642), ('세종전의', 0.37873043302897264), ('아산', 0.19793081346136326)],
      "예산군": [('예산', 0.5326095761484639), ('유구', 0.24550618666774907), ('아산', 0.221884237183787)],
      "이원면": [('태안', 0.38998768925871385), ('안도', 0.3455141387600907), ('대산', 0.2644981719811954)],
      "홍성읍": [('홍북', 0.7537093709927019), ('홍성죽도', 0.12481964368426245), ('예산', 0.1214709853230357)],
      "성성동": [('성거', 0.6985976183654422), ('세종전의', 0.17642014446434248), ('아산', 0.12498223717021528)]
    }
    self.device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    
    
  def setup(self):
    location = "공주"
    self.pm_df = pd.read_csv(f"./dataset/TEST_INPUT/{location}.csv")
    self.pm_df.drop(columns=["연도", "일시", "측정소"], inplace=True)
    
    aws_dfs = []
    aws_weights = []
    for name, weight in self.near_aws[location]:
      aws_dfs.append(pd.read_csv(f"./dataset/TEST_AWS/{name}.csv"))
      aws_weights.append(weight)
      
    cols = [np.zeros(len(self.pm_df)) for _ in range(5)]
    for aws_df, w in zip(aws_dfs, aws_weights):
      cols[0] += np.array(aws_df["기온(°C)"]) * w
      cols[1] += np.array(aws_df["풍향(deg)"]) * w
      cols[2] += np.array(aws_df["풍속(m/s)"]) * w
      cols[3] += np.array(aws_df["강수량(mm)"]) * w
      cols[4] += np.array(aws_df["습도(%)"]) * w
          
    self.pm_df["기온(°C)"] = cols[0]
    self.pm_df["풍향(deg)"] = cols[1]
    self.pm_df["풍속(m/s)"] = cols[2]
    self.pm_df["강수량(mm)"] = cols[3]
    self.pm_df["습도(%)"] = cols[4]
    
    self.pm_df["풍향(deg)"] = self.pm_df["풍향(deg)"].apply(lambda x: x * 359)
    wv = self.pm_df['풍속(m/s)'].values
    wd_rad = self.pm_df['풍향(deg)'].values * np.pi / 180
    self.pm_df['Wx'] = wv*np.cos(wd_rad)
    self.pm_df['Wy'] = wv*np.sin(wd_rad)
    self.pm_df.drop(columns=["풍향(deg)", "풍속(m/s)"], inplace=True)

    
    # model가져오기
    self.model = torch.load("./lstm.pth", map_location=self.device)
  
  def predict(self):
    X = torch.tensor(self.pm_df.iloc[:24*2].values).unsqueeze(0)
    X = X.float().to(self.device)
    output = self.model(X)
    
    
    return output

In [30]:
predictor = Predictor()
predictor.setup()

In [31]:
predictor.predict()

torch.Size([1, 48, 6])


tensor([[0.0600, 0.0515, 0.0434, 0.0382, 0.0293, 0.0265, 0.0217, 0.0174, 0.0155,
         0.0144, 0.0133, 0.0124, 0.0126, 0.0202, 0.0208, 0.0230, 0.0276, 0.0308,
         0.0309, 0.0310, 0.0358, 0.0400, 0.0479, 0.0549, 0.0596, 0.0601, 0.0608,
         0.0590, 0.0595, 0.0566, 0.0606, 0.0591, 0.0634, 0.0678, 0.0688, 0.0715,
         0.0781, 0.0761, 0.0804, 0.0805, 0.0818, 0.0802, 0.0795, 0.0777, 0.0836,
         0.0889, 0.0912, 0.0989, 0.0970, 0.1000, 0.0952, 0.0874, 0.0896, 0.0876,
         0.0872, 0.0923, 0.0922, 0.0916, 0.0933, 0.0982, 0.1052, 0.1038, 0.1100,
         0.1079, 0.1115, 0.1090, 0.1074, 0.1103, 0.1080, 0.1166, 0.1229, 0.1304]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

In [18]:
predictor.pm_df.head()

,연도,일시,측정소,PM2.5,기온(°C),강수량(mm),습도(%),Wx,Wy
0,4,01-01 00:00,공주,0.060,0.237713,0.0,0.651381,0.017606,0.027971
1,4,01-01 01:00,공주,0.064,0.222599,0.0,0.670922,0.017718,0.012799
2,4,01-01 02:00,공주,0.072,0.203204,0.0,0.730556,0.006023,0.004526
3,4,01-01 03:00,공주,0.064,0.195173,0.0,0.755246,0.001175,0.006596
4,4,01-01 04:00,공주,0.056,0.185312,0.0,0.796919,0.002558,0.000000
